## MNIST Training Example

In [1]:
import sys
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import wandb
sys.argv = ['mnist']

### Training Settings 

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64)
parser.add_argument('--test-batch-size', type=int, default=1000)
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--momentum', type=float, default=0.5)
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1)
parser.add_argument('--log-interval', type=int, default=10)
args = parser.parse_args()

In [3]:
# Setup W & B
wandb.init(
    name='MNIST', 
    config=dict(),  
    project='pytorch-demo', 
    tags=['improved'],
    dir='./runs',
    entity='awaelchli',  # change this to your username!
    group='mnist',
    resume=False,
)

# We load all of the arguments into config to save as hyperparameters
wandb.config.update(args)

### Choosing the Device

In [4]:
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if use_cuda else "cpu")

### Model Definition


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = Net().to(device)

In [6]:
# This magic line lets us save the pytorch model and track all of the gradients and optionally parameters
wandb.watch(model)

### Setup Data Loading

In [7]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
transform = transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('../data', train=False, transform=transform)
               
train_loader = torch.utils.data.DataLoader(train_data, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size, shuffle=True, **kwargs)

### Optimizer

In [8]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

### Training Loop

In [9]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            
            wandb.log({
                'Training Loss': loss.item(),
            })
            
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### Testing

In [10]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    example_images = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            # Save the first inbput tensor in each test batch as an example image
            example_images.append(wandb.Image(data[0], caption="Pred: {} Truth: {}".format(pred[0].item(), target[0])))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    # Log the images and metrics
    wandb.log({
        "Examples": example_images,
        "Test Accuracy": 100. * correct / len(test_loader.dataset),
        "Test Loss": test_loss
    })


### Main Loop


In [11]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332586
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.323807
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.262323
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.307054
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.248528
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.214086
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.219406
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.205292
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.088662
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.981406
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.929761
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.844702
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.841604
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.779582
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.768399
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.512483
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.443120
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.446421
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.384257
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.212498
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.222623
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.483124
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.258694
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.288998
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.271974
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.179445
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.203963
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.279617
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.325345
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.325360
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.223496
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.239686
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.299639
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.402223
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.336746
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.375958
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.332467
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.419760
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.307775
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.264303


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.178812
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.242396
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.383589
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.144584
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.279657
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.145718
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.445149
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.284926
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.563386
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.342596
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.326173
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.273419
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.156835
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.380382
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.309982
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.387320
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.233342
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.362507
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.129642
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.298375


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.261008
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.219291
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.239006
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.206614
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.235889
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.223366
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.184404
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.156328
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.222884
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.199011
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.195609
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.206329
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.205350
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.361909
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.310551
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.250739
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.433516
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.181695
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.188247
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.313828


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.226941
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.137736
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.142192
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.279761

Test set: Average loss: 0.0616, Accuracy: 9811/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.077940
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.104923
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.100989
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.061258
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.091201
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.254883
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.099860
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.143990
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.182936
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.314666
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.177112
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.095773
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.241903
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.151991
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.210423
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.195669
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.174242
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.083422
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.193106
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.145046
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.230645
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.126038
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.177299
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.174751
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.173408
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.209584
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.117608
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.130259
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.093922
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.226813
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.101504
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.191727
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.077002
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.156563
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.122288
